# Capitulo 1 - Introdução

Nesse capítulo serão apresentados conceitualmente os conceito centrais do trabalhos do cientista de dados a partir de uma base de dados gerada manualmente. Essa base se baseia em uma empresa fictícia que gerencia uma rede social voltada para cientistas de dados.

In [10]:
from collections import Counter, defaultdict

In [11]:
users = [
    {"id" : 0, "name" : "Hero"},
    {"id" : 1, "name" : "Dunn"},
    {"id" : 2, "name" : "Sue"},
    {"id" : 3, "name" : "Chi"},
    {"id" : 4, "name" : "Thor"},
    {"id" : 5, "name" : "Clive"},
    {"id" : 6, "name" : "Hicks"},
    {"id" : 7, "name" : "Devin"},
    {"id" : 8, "name" : "Kate"},
    {"id" : 9, "name" : "Klein"},
]

pares_de_conexoes = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [12]:
amizades = {user['id']: [] for user in users}
amizades                                                                                                                

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

In [13]:
# Lista todas as conexoes de cada usuário

for i, j in pares_de_conexoes:
    amizades[i].append(j)
    amizades[j].append(i)

amizades

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

In [14]:
# Conta quantas conexões existem na rede

def numero_de_amigos(user):
    user_id = user['id']
    friends_ids = amizades[user_id]
    return len(friends_ids)

total_de_conexoes = sum(numero_de_amigos(user)
                        for user in users)

# Calcula média de conexões por usuário

num_de_usuarios = len(users)
media_de_amizades = total_de_conexoes / num_de_usuarios
media_de_amizades

2.4

In [15]:
# Encontra o usuário com mais conexões

numero_de_amigos_por_id = [(user["id"], numero_de_amigos(user))
                            for user in users]

numero_de_amigos_por_id.sort(key = lambda id_e_amigos: id_e_amigos[1], reverse = True)
numero_de_amigos_por_id

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

In [16]:
# Usuários que talvez você conheça

def foaf_ids(user):
    return [foaf_id 
            for id_conexao in amizades[user["id"]]
            for foaf_id in amizades[id_conexao]]

print(amizades[1])
print(amizades[2])

print(foaf_ids(users[0]))

[0, 2, 3]
[0, 1, 3]
[0, 2, 3, 0, 1, 3]


In [17]:
def amigo_do_amigo(user):
    user_id = user["id"]
    return Counter(
        foaf_id
        for id_conexao in amizades[user_id]
        for foaf_id in amizades[id_conexao]
        if foaf_id != user_id and foaf_id not in amizades[user_id]
    )

print(amigo_do_amigo(users[3]))

Counter({0: 2, 5: 1})


In [18]:
# Cria uma separação por interesses para uma recomendação mais precisa

interesses = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodel"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial inteligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [19]:
def cientistas_de_dados_que_gostam(assunto):
    return[user_id
           for user_id, user_interest in interesses
           if user_interest == assunto]

users_ids_por_interesse = defaultdict(list)

for user_id, interesse in interesses:
    users_ids_por_interesse[interesse].append(user_id)

users_ids_por_interesse

defaultdict(list,
            {'Hadoop': [0, 9],
             'Big Data': [0, 8, 9],
             'HBase': [0, 1],
             'Java': [0, 5, 9],
             'Spark': [0],
             'Storm': [0],
             'Cassandra': [0, 1],
             'NoSQL': [1],
             'MongoDB': [1],
             'Postgres': [1],
             'Python': [2, 3, 5],
             'scikit-learn': [2, 7],
             'scipy': [2],
             'numpy': [2],
             'statsmodel': [2],
             'pandas': [2],
             'R': [3, 5],
             'statistics': [3, 6],
             'regression': [3, 4],
             'probability': [3, 6],
             'machine learning': [4, 7],
             'decision trees': [4],
             'libsvm': [4],
             'C++': [5],
             'Haskell': [5],
             'programming languages': [5],
             'mathematics': [6],
             'theory': [6],
             'Mahout': [7],
             'neural networks': [7, 8],
             'deep learning': [

In [20]:
interesses_por_user_id = defaultdict(list)

for user_id, interesse in interesses:
    interesses_por_user_id[user_id].append(interesse)

interesses_por_user_id

defaultdict(list,
            {0: ['Hadoop',
              'Big Data',
              'HBase',
              'Java',
              'Spark',
              'Storm',
              'Cassandra'],
             1: ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
             2: ['Python',
              'scikit-learn',
              'scipy',
              'numpy',
              'statsmodel',
              'pandas'],
             3: ['R', 'Python', 'statistics', 'regression', 'probability'],
             4: ['machine learning', 'regression', 'decision trees', 'libsvm'],
             5: ['Python',
              'R',
              'Java',
              'C++',
              'Haskell',
              'programming languages'],
             6: ['statistics', 'probability', 'mathematics', 'theory'],
             7: ['machine learning',
              'scikit-learn',
              'Mahout',
              'neural networks'],
             8: ['neural networks',
              'deep learning',
        

In [21]:
def assuntos_mais_comuns(user):
    return Counter(
        interesse_por_user_id
        for interesse in interesses_por_user_id[user['id']]
        for interesse_por_user_id in users_ids_por_interesse[interesse]
        if interesse_por_user_id != user['id']
    )

assuntos_mais_comuns(users[0])

Counter({9: 3, 1: 2, 8: 1, 5: 1})

In [22]:
salario_e_experiencia = [
    (83000, 8.7), (88000, 8.1),
    (48000, 0.7), (76000, 6),
    (69000, 6.5), (76000, 7.5),
    (60000, 2.5), (83000, 10),
    (48000, 1.9), (63000, 4.2)]